In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Input
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

file_path = '/content/drive/My Drive/data.csv'
data = pd.read_csv(file_path)

texts = data['Answer'].values
labels = data.iloc[:, 1:].astype(int).values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

input_ids = []
attention_masks = []

for text in texts:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=64,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='tf',
        truncation=True,
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

# Convert tensor to numpy array only if it is not already a numpy array
input_ids_np = input_ids.numpy() if not isinstance(input_ids, np.ndarray) else input_ids
attention_masks_np = attention_masks.numpy() if not isinstance(attention_masks, np.ndarray) else attention_masks

# Split the dataset into training and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids_np, labels, random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(
    attention_masks_np, np.zeros(attention_masks_np.shape[0]), random_state=2018, test_size=0.1)

bert_model = TFBertModel.from_pretrained('bert-base-uncased')

input_id_layer = Input(shape=(64,), dtype='int32')
input_mask_layer = Input(shape=(64,), dtype='int32')
embeddings = bert_model(input_id_layer, attention_mask=input_mask_layer).pooler_output
out = Dense(units=labels.shape[1], activation='sigmoid')(embeddings)

model = tf.keras.Model(inputs=[input_id_layer, input_mask_layer], outputs=out)

model.compile(optimizer=Adam(learning_rate=2e-5),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy(name='accuracy')])

history = model.fit(
    [train_inputs, train_masks],
    train_labels,
    batch_size=32,
    epochs=4,
    validation_data=([validation_inputs, validation_masks], validation_labels),
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
)

Mounted at /content/drive


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/4
42/42 [==============================] - 1146s 26s/step - loss: 0.4350 - accuracy: 0.8200 - val_loss: 0.2808 - val_accuracy: 0.9070
Epoch 2/4
42/42 [==============================] - 1070s 25s/step - loss: 0.2704 - accuracy: 0.8982 - val_loss: 0.2559 - val_accuracy: 0.9054
Epoch 3/4
42/42 [==============================] - 1061s 25s/step - loss: 0.2476 - accuracy: 0.9037 - val_loss: 0.2342 - val_accuracy: 0.9161
Epoch 4/4
42/42 [==============================] - 1056s 25s/step - loss: 0.2206 - accuracy: 0.9167 - val_loss: 0.2137 - val_accuracy: 0.9178


In [14]:

import os
from tensorflow.keras.models import load_model

# Replace 'Colab Models/MyTrainedModel' with your desired subdirectory in Google Drive
folder_path = '/content/drive/My Drive/Colab Models/colorassociation'
os.makedirs(folder_path, exist_ok=True)

# Now save the model to this directory
model.save(folder_path, save_format='tf')


In [22]:
from google.colab import drive
from transformers import BertTokenizer
import tensorflow as tf

# Mount Google Drive to access the saved model
drive.mount('/content/drive')

# Specify the path where your trained model is saved
model_path = '/content/drive/My Drive/Colab Models/colorassociation'

# Load the saved model from the path
model = tf.keras.models.load_model(model_path)

# Initialize the tokenizer used during training
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to encode texts for the model, this should match the training process
def prepare_text_for_model(text, tokenizer, max_length):
    encoded_dict = tokenizer.encode_plus(
        text,                      # Input text
        add_special_tokens=True,   # Add '[CLS]' and '[SEP]'
        max_length=max_length,     # Pad & truncate all sentences
        padding='max_length',
        return_attention_mask=True,# Construct attention masks
        return_tensors='tf',       # Return tf tensors
        truncation=True
    )

    # The model expects a list of tensors as input
    return [encoded_dict['input_ids'], encoded_dict['attention_mask']]

# Prepare an example input text
test_text = "Your example sentence here."

# Encode the example text using the same function used at training time
prepared_input = prepare_text_for_model(test_text, tokenizer, max_length=64)

# Predict using the model
prediction = model.predict(prepared_input)

# Print the raw prediction values
print(f"Raw prediction output: {prediction}")

# Apply a threshold to determine the class
threshold = 0.5
predicted_class = (prediction >= threshold).astype(int)

# Print the predicted class
print(f"Predicted class: {predicted_class}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: ignored